In [23]:
#imports
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from math import sqrt
import pandas as pd
import glob
from numpy import random
import sys
import pathlib
import math
from math import *
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [24]:
tracks_dataset = pd.DataFrame()
data_dir = pathlib.Path('data')
paths=list(data_dir.glob('*tracks.csv'))
merger = pd.DataFrame()
for x in paths: 
    #get 3 datasets
    data = pd.read_csv(x)
    str_add = str(x)
    num = str_add[5:7]
    meta = pd.read_csv('data/'+num+'_recordingMeta.csv')
    trackmeta = pd.read_csv('data/'+num+'_tracksMeta.csv')
    
    #merge tracks and tracksmeta datasets
    merger = pd.merge(data, trackmeta, on=['trackId'])
    
    #add and calculate additional columns
    frame_rate = float(meta['frameRate'])
    merger['frameRate'] = frame_rate
    
    #timestamp
    merger['current_time'] = round(merger['frame']/merger['frameRate'])
    
    #vehicle density at a given time
    merger['vehicle_density'] = merger.groupby(['current_time'])['trackId'].transform('nunique')
    
    #slip angle
    merger['slipangle'] = np.degrees(np.arctan(merger['latVelocity']/merger['lonVelocity']))
    merger = merger.fillna(0)
    
    #speed limit
    speedLimit = float(meta['speedLimit'])
    merger['speedLimit'] = speedLimit
    
    #duration of recording
    duration = float(meta['duration'])
    merger['duration'] = duration
    
    #number of tracks / objects of interest
    numTracks = int(meta['numTracks'])
    merger['numTracks'] = numTracks
    
    #number of vehicles
    numVehicles = float(meta['numVehicles'])
    merger['numVehicles'] = numVehicles   
    
    #number of vulnerable users (pedestrians, bikes, motercycles)
    numVRUs = int(meta['numVRUs'])
    merger['numVRUs'] = numVRUs
    
    #velocity
    merger['totalVelocity']=np.sqrt((merger['xVelocity']**2)+(merger['yVelocity']**2))
    
    #acceleration
    merger['totalAcceleration']=np.sqrt((merger['xAcceleration']**2)+(merger['yAcceleration']**2))
    
    #rounded number for position
    merger['xCenterRound'] = merger.xCenter.round()
    merger['yCenterRound'] = merger.yCenter.round()
    
    #potential collision = different vehicles found in approximate same position
    merger['num_collision_vehicles'] = merger.groupby(['recordingId_x','xCenterRound','yCenterRound','current_time'])['trackId'].transform('nunique')
    
    #add each of total 23 instances to final dataset
    tracks_dataset = tracks_dataset.append(merger,ignore_index=False)
    
    
tracks_dataset


,recordingId_x,trackId,frame,trackLifetime,xCenter,yCenter,heading,width_x,length_x,xVelocity,...,speedLimit,duration,numTracks,numVehicles,numVRUs,totalVelocity,totalAcceleration,xCenterRound,yCenterRound,num_collision_vehicles
0,0,0,25218,0,141.0003,-138.8970,107.7517,0.8158,1.0443,-0.2620,...,13.8889,1028.64,348,324.0,24,0.812502,0.518117,141.0,-139.0,1
1,0,0,25219,1,140.9896,-138.8657,107.7339,0.8158,1.0443,-0.2655,...,13.8889,1028.64,348,324.0,24,0.833326,0.527564,141.0,-139.0,1
2,0,0,25220,2,140.9791,-138.8335,107.7124,0.8158,1.0443,-0.2691,...,13.8889,1028.64,348,324.0,24,0.854480,0.536753,141.0,-139.0,1
3,0,0,25221,3,140.9686,-138.8011,107.6877,0.8158,1.0443,-0.2730,...,13.8889,1028.64,348,324.0,24,0.876214,0.545587,141.0,-139.0,1
4,0,0,25222,4,140.9579,-138.7678,107.6599,0.8158,1.0443,-0.2770,...,13.8889,1028.64,348,324.0,24,0.898561,0.553913,141.0,-139.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163982,23,463,23712,140,71.5383,-63.3552,332.0436,2.1015,4.5571,6.1692,...,13.8889,948.68,464,456.0,8,6.827621,1.897782,72.0,-63.0,1
163983,23,463,23713,141,71.7839,-63.4703,332.5928,2.1015,4.5571,6.2020,...,13.8889,948.68,464,456.0,8,6.828499,1.897009,72.0,-63.0,1
163984,23,463,23714,142,72.0310,-63.5823,333.1645,2.1015,4.5571,6.2356,...,13.8889,948.68,464,456.0,8,6.831107,1.896650,72.0,-64.0,1
163985,23,463,23715,143,72.2840,-63.6944,333.7808,2.1015,4.5571,6.2681,...,13.8889,948.68,464,456.0,8,6.833301,1.896595,72.0,-64.0,1


In [25]:
tracks_dataset.columns

Index(['recordingId_x', 'trackId', 'frame', 'trackLifetime', 'xCenter',
       'yCenter', 'heading', 'width_x', 'length_x', 'xVelocity', 'yVelocity',
       'xAcceleration', 'yAcceleration', 'lonVelocity', 'latVelocity',
       'lonAcceleration', 'latAcceleration', 'recordingId_y', 'initialFrame',
       'finalFrame', 'numFrames', 'width_y', 'length_y', 'class', 'frameRate',
       'current_time', 'vehicle_density', 'slipangle', 'speedLimit',
       'duration', 'numTracks', 'numVehicles', 'numVRUs', 'totalVelocity',
       'totalAcceleration', 'xCenterRound', 'yCenterRound',
       'num_collision_vehicles'],
      dtype='object')

In [26]:
for i in range(0,38):
    print(i,tracks_dataset.columns[i], type(tracks_dataset.iloc[i][0]))

0 recordingId_x <class 'numpy.int64'>
1 trackId <class 'numpy.int64'>
2 frame <class 'numpy.int64'>
3 trackLifetime <class 'numpy.int64'>
4 xCenter <class 'numpy.int64'>
5 yCenter <class 'numpy.int64'>
6 heading <class 'numpy.int64'>
7 width_x <class 'numpy.int64'>
8 length_x <class 'numpy.int64'>
9 xVelocity <class 'numpy.int64'>
10 yVelocity <class 'numpy.int64'>
11 xAcceleration <class 'numpy.int64'>
12 yAcceleration <class 'numpy.int64'>
13 lonVelocity <class 'numpy.int64'>
14 latVelocity <class 'numpy.int64'>
15 lonAcceleration <class 'numpy.int64'>
16 latAcceleration <class 'numpy.int64'>
17 recordingId_y <class 'numpy.int64'>
18 initialFrame <class 'numpy.int64'>
19 finalFrame <class 'numpy.int64'>
20 numFrames <class 'numpy.int64'>
21 width_y <class 'numpy.int64'>
22 length_y <class 'numpy.int64'>
23 class <class 'numpy.int64'>
24 frameRate <class 'numpy.int64'>
25 current_time <class 'numpy.int64'>
26 vehicle_density <class 'numpy.int64'>
27 slipangle <class 'numpy.int64'>
28 

In [27]:
tracks_dataset.loc[tracks_dataset['num_collision_vehicles']==2]

,recordingId_x,trackId,frame,trackLifetime,xCenter,yCenter,heading,width_x,length_x,xVelocity,...,speedLimit,duration,numTracks,numVehicles,numVRUs,totalVelocity,totalAcceleration,xCenterRound,yCenterRound,num_collision_vehicles
7423,0,11,17286,325,106.3828,-73.5690,277.3582,0.8868,2.0866,1.0037,...,13.8889,1028.64,348,324.0,24,6.102901,2.297965,106.0,-74.0,2
7424,0,11,17287,326,106.4242,-73.8107,277.7967,0.8868,2.0866,1.0854,...,13.8889,1028.64,348,324.0,24,6.164901,2.319336,106.0,-74.0,2
7449,0,11,17312,351,108.4926,-80.6023,286.4803,0.8868,2.0866,2.8954,...,13.8889,1028.64,348,324.0,24,8.135005,2.031497,108.0,-81.0,2
7470,0,11,17333,372,111.1702,-87.6007,289.4123,0.8868,2.0866,3.2736,...,13.8889,1028.64,348,324.0,24,9.624439,1.523455,111.0,-88.0,2
7471,0,11,17334,373,111.3011,-87.9641,289.4698,0.8868,2.0866,3.2711,...,13.8889,1028.64,348,324.0,24,9.682861,1.495545,111.0,-88.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140621,23,391,20133,343,111.8533,-60.8753,354.2856,2.2138,3.5207,8.5659,...,13.8889,948.68,464,456.0,8,8.579271,2.337121,112.0,-61.0,2
140622,23,391,20134,344,112.1985,-60.8962,353.7759,2.2138,3.5207,8.5969,...,13.8889,948.68,464,456.0,8,8.615683,2.339265,112.0,-61.0,2
140623,23,391,20135,345,112.5446,-60.9218,353.2743,2.2138,3.5207,8.6270,...,13.8889,948.68,464,456.0,8,8.652072,2.340264,113.0,-61.0,2
140624,23,391,20136,346,112.8903,-60.9514,352.7660,2.2138,3.5207,8.6570,...,13.8889,948.68,464,456.0,8,8.689186,2.340585,113.0,-61.0,2


In [28]:
#write finalized dataset to csv
tracks_dataset.to_csv("combined_dataset.csv", index=False)

In [29]:
#read finalized dataset as check
data = pd.read_csv('combined_dataset.csv')
data

,recordingId_x,trackId,frame,trackLifetime,xCenter,yCenter,heading,width_x,length_x,xVelocity,...,speedLimit,duration,numTracks,numVehicles,numVRUs,totalVelocity,totalAcceleration,xCenterRound,yCenterRound,num_collision_vehicles
0,0,0,25218,0,141.0003,-138.8970,107.7517,0.8158,1.0443,-0.2620,...,13.8889,1028.64,348,324.0,24,0.812502,0.518117,141.0,-139.0,1
1,0,0,25219,1,140.9896,-138.8657,107.7339,0.8158,1.0443,-0.2655,...,13.8889,1028.64,348,324.0,24,0.833326,0.527564,141.0,-139.0,1
2,0,0,25220,2,140.9791,-138.8335,107.7124,0.8158,1.0443,-0.2691,...,13.8889,1028.64,348,324.0,24,0.854480,0.536753,141.0,-139.0,1
3,0,0,25221,3,140.9686,-138.8011,107.6877,0.8158,1.0443,-0.2730,...,13.8889,1028.64,348,324.0,24,0.876214,0.545587,141.0,-139.0,1
4,0,0,25222,4,140.9579,-138.7678,107.6599,0.8158,1.0443,-0.2770,...,13.8889,1028.64,348,324.0,24,0.898561,0.553913,141.0,-139.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5447090,23,463,23712,140,71.5383,-63.3552,332.0436,2.1015,4.5571,6.1692,...,13.8889,948.68,464,456.0,8,6.827621,1.897782,72.0,-63.0,1
5447091,23,463,23713,141,71.7839,-63.4703,332.5928,2.1015,4.5571,6.2020,...,13.8889,948.68,464,456.0,8,6.828499,1.897009,72.0,-63.0,1
5447092,23,463,23714,142,72.0310,-63.5823,333.1645,2.1015,4.5571,6.2356,...,13.8889,948.68,464,456.0,8,6.831107,1.896650,72.0,-64.0,1
5447093,23,463,23715,143,72.2840,-63.6944,333.7808,2.1015,4.5571,6.2681,...,13.8889,948.68,464,456.0,8,6.833301,1.896595,72.0,-64.0,1


Safety Measurement
How to determine

    Vehicle Density: Check how many cars are on the road at a given time.

Obstacle Safety: Check for any foreign objects on the road that can impact safety (i.e., broken car pieces, broken glass, etc.).

Poor Weather: Check for poor weather (i.e.,  heavy rain, heavy snowing/ hail, fog, etc.).

Unusual Driving: Check for cars traveling on the wrong side of the road, or for reckless driving .     merger['unusualDriving']=np.where((merger['heading']>0 and merger['totalVelocity']>0)

    Driver Awareness: Check for cars stopped for an extended period of time.(velocity 0 and acceleration 0)
    
    Speed Limit Compliance: Check for cars traveling at speeds higher than the speed limit.

Road Size: Check that the roads are large enough to accommodate the amount of cars regularly traveling through.

    Collision Safety: Predict whether or not a car will collide with another car. (frame within 10, xCenter and yCenter within 5)

    Pedestrian Safety: Check to see if there are pedestrians regularly crossing the road.

Theft: Check to see if there are regular car thefts (i.e., robbery, carjacking, etc.).

Road Condition: Check to see if the road is in good enough condition to drive on (i.e., no potholes, no cracks in the asphalt, etc.).

    Slip Angle: the angle between the direction in which a wheel is pointing and the direction in which it is actually traveling

In [30]:
for i in range(0,38):
    print(i,tracks_dataset.columns[i], type(tracks_dataset.iloc[i][0]))

0 recordingId_x <class 'numpy.int64'>
1 trackId <class 'numpy.int64'>
2 frame <class 'numpy.int64'>
3 trackLifetime <class 'numpy.int64'>
4 xCenter <class 'numpy.int64'>
5 yCenter <class 'numpy.int64'>
6 heading <class 'numpy.int64'>
7 width_x <class 'numpy.int64'>
8 length_x <class 'numpy.int64'>
9 xVelocity <class 'numpy.int64'>
10 yVelocity <class 'numpy.int64'>
11 xAcceleration <class 'numpy.int64'>
12 yAcceleration <class 'numpy.int64'>
13 lonVelocity <class 'numpy.int64'>
14 latVelocity <class 'numpy.int64'>
15 lonAcceleration <class 'numpy.int64'>
16 latAcceleration <class 'numpy.int64'>
17 recordingId_y <class 'numpy.int64'>
18 initialFrame <class 'numpy.int64'>
19 finalFrame <class 'numpy.int64'>
20 numFrames <class 'numpy.int64'>
21 width_y <class 'numpy.int64'>
22 length_y <class 'numpy.int64'>
23 class <class 'numpy.int64'>
24 frameRate <class 'numpy.int64'>
25 current_time <class 'numpy.int64'>
26 vehicle_density <class 'numpy.int64'>
27 slipangle <class 'numpy.int64'>
28 